In [1]:
import numpy as np
import pandas as pd
import string
import math
import json
import pydotplus
from IPython.display import Image 

## Load the Lending Club Dataset

In [2]:
loans = pd.read_csv('lending-club-data.csv',low_memory=False)

In [3]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans.drop('bad_loans', axis=1, inplace=True)

In [4]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'

# Extract the feature columns and target column
loans = loans[features + [target]]

## One Hot Encoding & Training and Validation data creation

In [5]:
loans = pd.get_dummies(loans)

In [6]:
train_index = pd.read_json('module-5-assignment-2-train-idx.json')
train_index = list(train_index[0])

In [7]:
test_index = pd.read_json('module-5-assignment-2-test-idx.json')
test_index = list(test_index[0])

In [8]:
train_data = loans.iloc[train_index]
test_data = loans.iloc[test_index]

In [9]:
print('Training set: %d data points' % len(train_data))
print('Test set: %d data points' % len(test_data))

Training set: 37224 data points
Test set: 9284 data points


In [10]:
train_data.safe_loans.value_counts(normalize=True)

 1    0.503654
-1    0.496346
Name: safe_loans, dtype: float64

In [11]:
test_data.safe_loans.value_counts(normalize=True)

-1    0.503447
 1    0.496553
Name: safe_loans, dtype: float64

## Decision tree implementation

### Function to count number of mistakes while predicting majority class

In [12]:
def intermediate_node_num_mistakes(labels_in_node):
    # Corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0    
    # Count the number of 1's (safe loans)
    ones = np.sum(np.where(labels_in_node==1,1,0))   
    # Count the number of -1's (risky loans)
    negative_ones = np.sum(np.where(labels_in_node==-1,1,0))          
    # Return the number of mistakes that the majority classifier makes.
    if ones > negative_ones:
        return negative_ones
    elif ones < negative_ones:
        return ones
    else:
        raise ValueError('Classes are equal')    

In [13]:
# Test case 1
example_labels = np.array([-1, -1, 1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print('Test passed!')
else:
    print('Test 1 failed... try again!')

# Test case 2
example_labels = np.array([-1, -1, 1, 1, 1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print('Test passed!')
else:
    print('Test 3 failed... try again!')
    
# Test case 3
example_labels = np.array([-1, -1, -1, -1, -1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print('Test passed!')
else:
    print('Test 3 failed... try again!')

Test passed!
Test passed!
Test passed!


### Function to pick best feature to split on

In [16]:
def best_splitting_feature(data, features, target):
    
    target_values = data[target]
    best_feature = None # Keep track of the best feature 
    best_error = 10     # Keep track of the best error so far 
    # Note: Since error is always <= 1, we should intialize it with something larger than 1.

    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(data))  
    
    # Loop through each feature to consider splitting on that feature
    for feature in features:
        
        # The left split will have all data points where the feature value is 0
        left_split = data[data[feature] == 0]
        
        # The right split will have all data points where the feature value is 1
        ## YOUR CODE HERE
        right_split =  data[data[feature] == 1]
            
        # Calculate the number of misclassified examples in the left split.
        # Remember that we implemented a function for this! (It was called intermediate_node_num_mistakes)
        # YOUR CODE HERE
        left_mistakes = intermediate_node_num_mistakes(left_split[target].as_matrix())

        # Calculate the number of misclassified examples in the rights split.
        ## YOUR CODE HERE
        right_mistakes = intermediate_node_num_mistakes(right_split[target].as_matrix())
            
        # Compute the classification error of this split.
        # Error = (# of mistakes (left) + # of mistakes (right)) / (# of data points)
        ## YOUR CODE HERE
        error = (left_mistakes + right_mistakes) / num_data_points

        # If this is the best error we have found so far, store the feature as best_feature and the error as best_error
        ## YOUR CODE HERE
        if error < best_error:
            best_error=error
            best_feature=feature
        
    return best_feature # Return the best feature we found

In [32]:
if best_splitting_feature(train_data, feature_set, 'safe_loans') == 'term_ 36 months':
    print('Test passed!')
else:
    print('Test failed... try again!')

Test passed!


### Building the tree

In [ ]:
def create_leaf(target_values):    
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True}   ## YOUR CODE HERE 
   
    # Count the number of data points that are +1 and -1 in this node.
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])    

    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] = 1        ## YOUR CODE HERE
    else:
        leaf['prediction'] = -1       ## YOUR CODE HERE        

    # Return the leaf node
    return leaf 

In [ ]:
def decision_tree_create(data, features, target, current_depth = 0, max_depth = 10):
    remaining_features = features[:] # Make a copy of the features.
    
    target_values = data[target]
    print "--------------------------------------------------------------------"
    print "Subtree, depth = %s (%s data points)." % (current_depth, len(target_values))
    

    # Stopping condition 1
    # (Check if there are mistakes at current node.
    # Recall you wrote a function intermediate_node_num_mistakes to compute this.)
    if  == 0:  ## YOUR CODE HERE
        print "Stopping condition 1 reached."     
        # If not mistakes at current node, make current node a leaf node
        return create_leaf(target_values)
    
    # Stopping condition 2 (check if there are remaining features to consider splitting on)
    if remaining_features == :   ## YOUR CODE HERE
        print "Stopping condition 2 reached."    
        # If there are no remaining features to consider, make current node a leaf node
        return create_leaf(target_values)    
    
    # Additional stopping condition (limit tree depth)
    if current_depth >= :  ## YOUR CODE HERE
        print "Reached maximum depth. Stopping for now."
        # If the max tree depth has been reached, make current node a leaf node
        return create_leaf(target_values)

    # Find the best splitting feature (recall the function best_splitting_feature implemented above)
    ## YOUR CODE HERE

    
    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    right_split =       ## YOUR CODE HERE
    remaining_features.remove(splitting_feature)
    print "Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split))
    
    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        print "Creating leaf node."
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        print "Creating leaf node."
        ## YOUR CODE HERE

        
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, current_depth + 1, max_depth)        
    ## YOUR CODE HERE
    right_tree = 

    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}